# ***P242***

In [1]:
import pandas as pd
wine=pd.read_csv('http://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)#검증세트 테스트 세트를 2번 나눈다고 보면 됨
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [5]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))
from sklearn.model_selection import cross_validate#cross_validata()함수 테스트 세트로 나눈후 디폴드값:5만큼 나눈 데이터들을 갖고 교차하며 훈련함 그중 최고값을 리턴해주는 교차 검증함수
scores=cross_validate(dt,train_input,train_target)
print(scores)
print(np.mean(scores['test_score']))


0.9971133028626413
0.864423076923077
{'fit_time': array([0.00742602, 0.007303  , 0.00759816, 0.0073483 , 0.00724649]), 'score_time': array([0.00077963, 0.00078297, 0.0007658 , 0.00077295, 0.00078678]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [7]:
from sklearn.model_selection import StratifiedKFold
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)#cv값은 폴드 교차 검증하는 회수를 정할수 있는 하이퍼파라미터 중 하나
print(np.mean(scores['test_score']))


0.8574181117533719


In [9]:
from sklearn.model_selection import GridSearchCV#위에 했던 검증세트 만들기와 폴드 교차검증하는것까지를 반복하며 실행해주는  사이킷런의 클래스  디폴드 cv값은 5
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=1)
gs.fit(train_input,train_target)
dt=gs.best_estimator_
print(dt.score(train_input,train_target))
print(gs.cv_results_['mean_test_score'])
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])#최적의 값을 도출해내는 모델의 하이퍼파라미터값을 리턴해줌

0.9615162593804117
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [10]:
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}#다양한 하이퍼파라미터 값을 정해주어 해당 값들마다 달라지는 최적의 값에 대한 모델을 찾을수 있고 그 최적값을 내는 하이퍼파라미터도 도출할수 있음
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)
print(gs.best_params_)#최적값을 도출해내는 하이퍼 파라미터값들

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [11]:
print(np.max(gs.cv_results_['mean_test_score']))


0.8683865773302731


In [18]:
from scipy.stats import uniform,randint
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25)}#하이퍼파라미터 값들을 랜덤으로 n_iter의 수만큼 반복하여 최적값을 찾아냄
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
rs.fit(train_input,train_target)
print(rs.best_params_)
print(np.max(rs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [19]:
dt=rs.best_estimator_#최적의 모델값이 저장됨
print(dt.score(test_input,test_target))

0.86


***P262 확인문제 3번***

In [21]:
from scipy.stats import uniform,randint
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25)}
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(DecisionTreeClassifier(splitter='random',random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
rs.fit(train_input,train_target)
print(rs.best_params_)
print(np.max(rs.cv_results_['mean_test_score']))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
